$$p_{t+1} = p_t + \Delta p_t$$
$$n_{t+1} = n_t + \Delta n_t = n_t + \{f(p_{t+1}) - n_t\} = f(p_{t+1})$$
$$R_{t+1} = p_{t+1} \times n_{t+1}$$
$$f(p_{t+1}) = f(p_t + \Delta p_t)$$
$$\Delta p_t = \{{\arg\max}_{p_t} p_t \times f(p_t)\} - p_t$$